# minst_cnn

- 딥러닝을 학습시키는 단계
- cnn 구조 확인
- MNIST에 cnn 적용시키는 방법

## 학습 단계

1. 라이브러리 가져오기
2. GPU 사용 설정, random value를 위한 seed
3. parameter설정
4. 데이터셋 가져오기
5. 학습모델 만들기
6. Loss function & 최적화도구 선택
7. 모델 학습 및 loss check
8. 모델 성능 확인

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed = 777

cpu


In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(
    root='MNIST_data/',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

mnist_test = dsets.MNIST(
    root='MNIST_data/',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [6]:
data_loader = torch.utils.data.DataLoader(
    dataset=mnist_test,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

In [7]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [8]:
model = CNN().to(device)

In [9]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [10]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch:{} cost = {}'.format(epoch + 1, avg_cost))
    
print('Learning finished')
        

Epoch:1 cost = 0.6738463640213013
Epoch:2 cost = 0.17841598391532898
Epoch:3 cost = 0.10339967161417007
Epoch:4 cost = 0.07112696766853333
Epoch:5 cost = 0.05583443120121956
Epoch:6 cost = 0.040757518261671066
Epoch:7 cost = 0.029781123623251915
Epoch:8 cost = 0.02401765249669552
Epoch:9 cost = 0.01937095634639263
Epoch:10 cost = 0.013794617727398872
Epoch:11 cost = 0.011639696545898914
Epoch:12 cost = 0.011626743711531162
Epoch:13 cost = 0.006786367855966091
Epoch:14 cost = 0.004223864525556564
Epoch:15 cost = 0.0057669514790177345
Learning finished


In [15]:
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    
    accuracy = correct_prediction.float().mean()
    
    print('Accuracy : ', accuracy.item())

c:\Users\muhayuleejanghee\Desktop\study\.venv\Lib\site-packages\torchvision\datasets\mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\muhayuleejanghee\Desktop\study\.venv\Lib\site-packages\torchvision\datasets\mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy :  0.996999979019165


## 테스트 결과

- 99.69%
- 여기에서 레이어 3와 레이어 4, 5를 추가하면 어떨까? 

![internal covariate shift.png](/python/pytorch_study/images/mnist_cnn_layer_add.png)


In [18]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625, bias=True)
        
        self.relu = nn.ReLU()
        
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out
    
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch:{} cost = {}'.format(epoch + 1, avg_cost))
    
print('Learning finished')

with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    
    accuracy = correct_prediction.float().mean()
    
    print('Accuracy : ', accuracy.item())
        

Epoch:1 cost = 0.514384925365448
Epoch:2 cost = 0.10562222450971603
Epoch:3 cost = 0.052530936896800995
Epoch:4 cost = 0.043791212141513824
Epoch:5 cost = 0.029839470982551575
Epoch:6 cost = 0.020164547488093376
Epoch:7 cost = 0.007961878553032875
Epoch:8 cost = 0.011308752000331879
Epoch:9 cost = 0.014625041745603085
Epoch:10 cost = 0.005085976794362068
Epoch:11 cost = 0.0024458442348986864
Epoch:12 cost = 0.0006160424672998488
Epoch:13 cost = 0.00017724769713822752
Epoch:14 cost = 0.0001121346140280366
Epoch:15 cost = 8.407561836065724e-05
Learning finished


c:\Users\muhayuleejanghee\Desktop\study\.venv\Lib\site-packages\torchvision\datasets\mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\muhayuleejanghee\Desktop\study\.venv\Lib\site-packages\torchvision\datasets\mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy :  0.9995999932289124
